<h1 style="text-align: center;">Project No. 1 - N-Grams & Sentiment Analysis</h1>

<h4 style="text-align: center;">

Esteban Gomez Valerio

Roi Jared Flores Garza Stone

Rafael Takata Garcia

Text Mining - O2025_MAF3654H

Ing. Juan Antonio Vega Fernández, M. Sc., M. T. Ed

ITESO
</h4>

#### Libraries

In [ ]:
import torchtext


#### Dataset

In [ ]:
imdb_data =  torchtext.datasets.IMDB